In [1]:
# === IMPORTS BÁSICOS E CONFIGURAÇÃO INICIAL ===

from pathlib import Path                     # Manipulação de caminhos de arquivos/pastas
import pandas as pd                           # Tabelas e manipulação de dados
import numpy as np                            # Utilidades numéricas
import fastf1                                 # API de dados de F1
from fastf1 import plotting                   # (opcional) utilidades extras do pacote

# === DEFINIÇÃO DE DIRETÓRIOS DO PROJETO ===
PROJ = Path.cwd().resolve().parent            # Diretório atual (raiz do projeto, se você abriu o notebook de lá)
DATA_DIR = PROJ / "data"                      # Pasta principal de dados
DATA_DIR.mkdir(exist_ok=True)                 # Garante que a pasta data exista
(DATA_DIR / "interim").mkdir(parents=True, exist_ok=True)   # Garante subpasta para dados intermediários

# === ATIVA CACHÊ LOCAL DO FASTF1 ===
CACHE_DIR = PROJ / "fastf1_cache"             # Pasta para cache do FastF1
CACHE_DIR.mkdir(exist_ok=True)                # Cria a pasta se não existir
fastf1.Cache.enable_cache(str(CACHE_DIR))     # Ativa o cache para acelerar chamadas futuras

# (opcional) estilos de plot do fastf1
plotting.setup_mpl(misc_mpl_mods=False)



c:\Users\pedro\iCloudDrive\Desktop\Eng. Elétrica UNESP Bauru 2020\TG\TG2\f1-tyre-strategy-simulator\.venv\Lib\site-packages\fastf1\plotting\_plotting.py:57: FutureWarning: The `misc_mpl_mods` argument was dropped from `.setup_mpl()` in version 3.6.0 and has no effect anymore. It will be removed in a future version of FastF1.
  warnings.warn(


In [8]:
# === FUNÇÕES AUXILIARES REUTILIZÁVEIS ===

def to_seconds(td):
    """Converte um pandas.Timedelta (ou NaT) para float em segundos."""
    return np.nan if pd.isna(td) else td.total_seconds()

def ensure_tyre_age(df: pd.DataFrame) -> pd.DataFrame:
    """Garante a coluna TyreLife; se ausente ou toda NaN, reconstrói por (Driver, Stint)."""
    df = df.copy()
    if "TyreLife" not in df.columns or df["TyreLife"].isna().all():
        # Ordena por piloto, stint e número da volta
        df = df.sort_values(["Driver", "Stint", "LapNumber"])
        # Conta cumulativamente as voltas dentro de cada stint (1,2,3,...)
        df["TyreLife"] = df.groupby(["Driver", "Stint"]).cumcount() + 1
    return df

def extract_event_laps(year: int, event_name: str) -> pd.DataFrame:
    """
    Extrai TODAS as voltas da CORRIDA (Race, 'R') para um (ano, evento) específico.
    Mantém in/out laps marcadas; converte tempos; agrega clima via merge_asof.
    Retorna um DataFrame pronto para concatenação posterior.
    """
    # Carrega a sessão de corrida do ano/evento usando FastF1
    session = fastf1.get_session(year, event_name, "R")
    session.load()  # Baixa dados (1ª vez) e usa cache nas próximas

    # Obtém todas as voltas como DataFrame
    laps = session.laps.reset_index(drop=True).copy()

    # Garante que colunas de pit existam mesmo se ausentes em alguma temporada
    for c in ["PitInTime", "PitOutTime"]:
        if c not in laps.columns:
            laps[c] = pd.NaT

    # Cria flags de in-lap/out-lap e uma flag agregada
    laps["IsPitInLap"]   = laps["PitInTime"].notna().astype(int)
    laps["IsPitOutLap"]  = laps["PitOutTime"].notna().astype(int)
    laps["IsInOrOutLap"] = (laps["IsPitInLap"] | laps["IsPitOutLap"]).astype(int)

    # Converte tempos de volta e setores para segundos mantendo o timedelta original
    for c in ["LapTime", "Sector1Time", "Sector2Time", "Sector3Time"]:
        if c in laps.columns:
            laps[c + "Sec"] = laps[c].apply(to_seconds)

    # Garante idade do pneu (TyreLife)
    laps = ensure_tyre_age(laps)

    # Adiciona metadados úteis
    laps["Year"] = year
    laps["EventName"] = event_name
    laps["Session"] = "Race"

    # Junta dados de clima da sessão (merge pelo tempo)
    weather = session.weather_data.copy()
    if weather is not None and not weather.empty and "Time" in weather.columns:
        # Alinha tipos para merge_asof (ambos como Timedelta)
        tmp = laps.copy()
        tmp["Time"] = pd.to_timedelta(tmp["Time"])
        weather["Time"] = pd.to_timedelta(weather["Time"])
        # Faz merge_asof pegando o valor de clima mais próximo no passado
        laps = pd.merge_asof(
            tmp.sort_values("Time"),
            weather[["Time","AirTemp","TrackTemp","Humidity","Pressure","WindSpeed","WindDirection"]].sort_values("Time"),
            on="Time", direction="backward"
        )

    # Ordena por piloto e volta para deixar o dataset “limpo”
    laps = laps.sort_values(["Driver", "LapNumber"]).reset_index(drop=True)
    return laps


In [9]:
# === ENTRADA DO USUÁRIO: INTERVALO DE ANOS E LISTAGEM DE CIRCUITOS ===

# Solicita ao usuário o ano inicial
year_start = int(input("Digite o ano INICIAL (ex.: 2018): ").strip())
# Solicita ao usuário o ano final
year_end   = int(input("Digite o ano FINAL   (ex.: 2024): ").strip())

# Garante que o intervalo esteja em ordem crescente
if year_end < year_start:
    year_start, year_end = year_end, year_start

# Cria uma lista de anos do intervalo (inclusive)
YEARS = list(range(year_start, year_end + 1))
print(f"Intervalo de anos selecionado: {YEARS}")

# Para descobrir quais eventos existem nesses anos, consultamos o calendário anual do FastF1
all_events = []                                 # Lista para acumular DataFrames de calendários
for y in YEARS:
    cal = fastf1.get_event_schedule(y, include_testing=False)  # Calendário oficial sem testes
    cal["Year"] = y                                           # Marca o ano no calendário
    all_events.append(cal[["Year","EventName","OfficialEventName","EventDate","Location"]])

# Concatena todos os calendários em um único DataFrame
events_df = pd.concat(all_events, ignore_index=True)

# Levanta a lista de eventos únicos que aparecem em pelo menos um dos anos
unique_events = sorted(events_df["EventName"].dropna().unique().tolist())

# Exibe uma lista enumerada para o usuário escolher o circuito
print("\n=== Circuitos com corridas válidas no intervalo ===")
for i, name in enumerate(unique_events, start=1):
    print(f"{i:2d}. {name}")

# Solicita ao usuário que escolha um índice da lista
idx = int(input("\nDigite o número do circuito desejado: ").strip())
# Valida o índice informado
if not (1 <= idx <= len(unique_events)):
    raise ValueError("Índice inválido. Rode novamente e escolha um número listado.")

# Obtém o nome do circuito selecionado
SELECTED_EVENT = unique_events[idx - 1]
print(f"\nVocê selecionou: {SELECTED_EVENT}")


Intervalo de anos selecionado: [2022, 2023, 2024]


C:\Users\pedro\AppData\Local\Temp\ipykernel_13720\2112252472.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cal["Year"] = y                                           # Marca o ano no calendário
C:\Users\pedro\AppData\Local\Temp\ipykernel_13720\2112252472.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cal["Year"] = y                                           # Marca o ano no calendário



=== Circuitos com corridas válidas no intervalo ===
 1. Abu Dhabi Grand Prix
 2. Australian Grand Prix
 3. Austrian Grand Prix
 4. Azerbaijan Grand Prix
 5. Bahrain Grand Prix
 6. Belgian Grand Prix
 7. British Grand Prix
 8. Canadian Grand Prix
 9. Chinese Grand Prix
10. Dutch Grand Prix
11. Emilia Romagna Grand Prix
12. French Grand Prix
13. Hungarian Grand Prix
14. Italian Grand Prix
15. Japanese Grand Prix
16. Las Vegas Grand Prix
17. Mexico City Grand Prix
18. Miami Grand Prix
19. Monaco Grand Prix
20. Qatar Grand Prix
21. Saudi Arabian Grand Prix
22. Singapore Grand Prix
23. Spanish Grand Prix
24. São Paulo Grand Prix
25. United States Grand Prix


C:\Users\pedro\AppData\Local\Temp\ipykernel_13720\2112252472.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cal["Year"] = y                                           # Marca o ano no calendário



Você selecionou: Bahrain Grand Prix


In [10]:
# === EXTRAÇÃO EM LOTE: PARA CADA ANO DO INTERVALO, COLETA AS VOLTAS DO EVENTO ESCOLHIDO ===

all_laps_list = []                               # Lista para acumular os DataFrames de cada ano
errors = []                                       # Lista para registrar anos que por algum motivo falharam

for y in YEARS:
    try:
        # Tenta extrair as voltas do evento selecionado naquele ano
        df_year = extract_event_laps(y, SELECTED_EVENT)
        # Acrescenta o DataFrame obtido na lista
        all_laps_list.append(df_year)
        # Log simples de sucesso
        print(f"[OK] {SELECTED_EVENT} {y}: {len(df_year)} voltas extraídas.")
    except Exception as e:
        # Em caso de erro (evento pode não ter corrida em algum ano), registra e segue
        print(f"[ERRO] {SELECTED_EVENT} {y}: {e}")
        errors.append((y, str(e)))

# Concatena tudo em um único DataFrame (se houver pelo menos um sucesso)
if all_laps_list:
    laps_all_years = pd.concat(all_laps_list, ignore_index=True)
    print(f"\nTotal de voltas combinadas: {len(laps_all_years)}")
else:
    raise RuntimeError("Nenhuma corrida foi extraída. Verifique o intervalo e o circuito selecionado.")


core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.6.1]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No ca

[OK] Bahrain Grand Prix 2022: 1125 voltas extraídas.


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!

[OK] Bahrain Grand Prix 2023: 1056 voltas extraídas.


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!

[OK] Bahrain Grand Prix 2024: 1129 voltas extraídas.

Total de voltas combinadas: 3310


In [ ]:
df = laps_raw.copy()

# cria colunas de pit se não existirem
for c in ["PitInTime","PitOutTime"]:
    if c not in df.columns:
        df[c] = pd.NaT

df["IsPitInLap"]   = df["PitInTime"].notna().astype(int)
df["IsPitOutLap"]  = df["PitOutTime"].notna().astype(int)
df["IsInOrOutLap"] = (df["IsPitInLap"] | df["IsPitOutLap"]).astype(int)

# converter tempos (mantém timedelta original + coluna em segundos)
for c in ["LapTime","Sector1Time","Sector2Time","Sector3Time"]:
    if c in df.columns:
        df[c+"Sec"] = df[c].apply(to_seconds)

# garantir idade de pneu
df = ensure_tyre_age(df)

# metadados
df["Year"] = YEAR
df["EventName"] = EVENT
df["Session"] = "Race"

display(df[["Driver","LapNumber","Stint","Compound","TyreLife","LapTime","LapTimeSec","IsInOrOutLap"]].head(10))
print(df.shape)


In [ ]:
weather = session.weather_data.copy()

# alinhar tipos para merge_asof
df_w = df.copy()
df_w["Time"] = pd.to_timedelta(df_w["Time"])
weather["Time"] = pd.to_timedelta(weather["Time"])

df_w = pd.merge_asof(
    df_w.sort_values("Time"),
    weather[["Time","AirTemp","TrackTemp","Humidity","Pressure","WindSpeed","WindDirection"]].sort_values("Time"),
    on="Time", direction="backward"
).sort_values(["Driver","LapNumber"]).reset_index(drop=True)

display(df_w.head(3))


In [ ]:
out_csv = DATA_DIR/"interim"/"bahrain_2022_all_laps.csv"
out_parq = DATA_DIR/"interim"/"bahrain_2022_all_laps.parquet"

df_w.to_csv(out_csv, index=False)
df_w.to_parquet(out_parq, index=False)

out_csv, out_parq, len(df_w)


In [ ]:
clean = df_w[(df_w["IsInOrOutLap"] == 0) & (df_w["Compound"].isin(["SOFT","MEDIUM","HARD"]))].copy()

import matplotlib.pyplot as plt
_ = clean.boxplot(column="LapTimeSec", by="Compound")
plt.title(f"Bahrein {YEAR} - Tempo por composto (voltas limpas)")
plt.suptitle("")
plt.xlabel("Composto")
plt.ylabel("LapTime (s)")
plt.show()

clean.groupby("Compound")["LapTimeSec"].describe()
